In [15]:
# === Dataset source: merged CSV (Jupyter & Streamlit compatible) ===
import altair as alt
import re
from pathlib import Path
import pandas as pd, numpy as np
import streamlit as st

# 1) BASE 경로: .py 파일이면 __file__, 노트북이면 CWD 사용
try:
    BASE = Path(__file__).parent
except NameError:
    BASE = Path.cwd()

# 2) 후보 경로에서 CSV 검색 (존재하는 첫 파일 사용)
CANDIDATES = [
    BASE / "spotify_merged.csv",                          # 권장: 루트에 두기
    BASE / "data" / "raw" / "spotify_merged.csv",
    BASE / "spotify_cleaned_final_v2.csv",                # 백업: 정제 CSV (있다면)
    BASE / "data" / "raw" / "spotify_cleaned_final_v2.csv",
]
CSV_PATH = next((p for p in CANDIDATES if p.exists()), None)
if CSV_PATH is None:
    st.error("❌ merged CSV를 찾지 못했습니다. `spotify_merged.csv`를 프로젝트 루트(또는 data/raw)에 두세요.")
    st.stop()

@st.cache_data(show_spinner=False)
def load_merged_csv(path: Path):
    """원본+매출지표 병합 CSV 로더
    - df_raw: 원본 그대로 (KPI/프리뷰/결측치 표시에 사용)
    - df: 차트용 작업본 (원본 열수에 영향 없이 파생만 추가)
    """
    df_raw = pd.read_csv(path)

    # 작업본 복사
    dfx = df_raw.copy()

    # revenue_num: 문자→숫자 (₩, 콤마 제거)
    if "revenue" in dfx.columns:
        def to_number(x):
            if pd.isna(x): return np.nan
            s = re.sub(r"[^0-9.\-]", "", str(x))
            return float(s) if s else np.nan
        dfx["revenue_num"] = dfx["revenue"].map(to_number)

    # 요금제 표준화 컬럼
    plan_col = next((c for c in ["subscription_plan","spotify_subscription_plan","subscription_plan_norm"]
                     if c in dfx.columns), None)
    if plan_col and "subscription_plan_norm" not in dfx.columns:
        dfx["subscription_plan_norm"] = dfx[plan_col].astype(str)

    # month 문자열화(Altair 정렬 일관성)
    if "month" in dfx.columns:
        dfx["month"] = dfx["month"].astype(str)

    return df_raw, dfx

# 3) 실제 로드 (여기서 반드시 df_raw/df를 생성해야 아래에서 사용 가능)
try:
    df_raw, df = load_merged_csv(CSV_PATH)
except Exception as e:
    st.error(f"데이터 로드 실패: {e}")
    st.stop()

# 4) 디버깅/확인용 캡션 (원하면 지워도 됨)
st.caption(f"✅ CSV 로드: {CSV_PATH.name} — {len(df_raw):,}행 × {len(df_raw.columns)}열 (원본 기준)")

2025-10-24 23:15:34.653 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-24 23:15:34.654 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-24 23:15:34.674 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-24 23:15:34.714 
  command:

    streamlit run /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-10-24 23:15:34.715 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-24 23:15:34.715 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [9]:
# ✅ 데이터 불러오기
df = pd.read_csv(RAW_FILE)
rows, cols = df.shape
print(f"불러온 데이터: {rows}행 × {cols}열")
df.head()

불러온 데이터: 3120행 × 25열


,userid,month,revenue,subscription_plan,timestamp,Age,Gender,spotify_usage_period,spotify_listening_device,spotify_subscription_plan,...,music_Influencial_mood,music_lis_frequency,music_expl_method,music_recc_rating,pod_lis_frequency,fav_pod_genre,preffered_pod_format,pod_host_preference,preffered_pod_duration,pod_variety_satisfaction
0,1,2023-01,60000,Premium,2023-01-01,20-35,Female,More than 2 years,Smart speakers or voice assistants,Premium (paid subscription),...,Sadness or melancholy,leisure time,Playlists,3,Daily,Comedy,Interview,Both,Both,Neutral
1,2,2023-01,60000,Premium,2023-01-01,12-20,Male,More than 2 years,Computer or laptop,Premium (paid subscription),...,Social gatherings or parties,Workout session,Playlists,2,Several times a week,Comedy,Interview,Both,No preference / Not applicable,Satisfied
2,3,2023-01,60000,Premium,2023-01-01,35-60,Others,6 months to 1 year,Smart speakers or voice assistants,Premium (paid subscription),...,Relaxation and stress relief,"Study Hours, While Traveling",Playlists,4,Once a week,Sports,Interview,No preference / Not applicable,Both,Satisfied
3,4,2023-01,60000,Premium,2023-01-01,20-35,Female,1 year to 2 years,"Smartphone, Smart speakers or voice assistants",Premium (paid subscription),...,"Relaxation and stress relief, Social gathering...","Office hours, Workout session, leisure time","recommendations, Playlists",4,Never,No preference / Not applicable,No preference / Not applicable,No preference / Not applicable,No preference / Not applicable,Neutral
4,5,2023-01,60000,Premium,2023-01-01,20-35,Female,1 year to 2 years,Smartphone,Premium (paid subscription),...,Relaxation and stress relief,leisure time,"recommendations, Playlists",4,Rarely,Lifestyle and Health,Story telling,Well known individuals,Both,Neutral


In [10]:
# revenue → 숫자 변환
def to_number(x):
    if pd.isna(x): return np.nan
    s = re.sub(r"[^0-9.\-]", "", str(x))
    return float(s) if s else np.nan

df["revenue_num"] = df["revenue"].apply(to_number)

# 날짜 처리
df["month"] = df["month"].astype(str)
df["month_key"] = df["month"].str.replace("-", "").astype(int)
df["timestamp_dt"] = pd.to_datetime(df.get("timestamp"), errors="coerce")

# 플랜 표준화
plan_col = next((c for c in ["subscription_plan","spotify_subscription_plan"] if c in df.columns), None)
df["subscription_plan_norm"] = df[plan_col].astype(str) if plan_col else "Unknown"

# ✅ 정합성 체크
checks = {
    "rows": len(df),
    "distinct_users": int(df["userid"].nunique()),
    "duplicates": int(df.duplicated(subset=["userid","month"]).sum()),
    "na_top5": df.isna().sum().sort_values(ascending=False).head(5).to_dict(),
    "month_range": {"min": df["month"].min(), "max": df["month"].max()},
    "plan_counts": df["subscription_plan_norm"].value_counts().to_dict()
}

print(json.dumps(checks, indent=2, ensure_ascii=False))

# 저장
with open(ART_MET / "summary.json", "w", encoding="utf-8") as f:
    json.dump(checks, f, ensure_ascii=False, indent=2)


{
  "rows": 3120,
  "distinct_users": 520,
  "duplicates": 0,
  "na_top5": {
    "userid": 0,
    "music_Influencial_mood": 0,
    "timestamp_dt": 0,
    "month_key": 0,
    "revenue_num": 0
  },
  "month_range": {
    "min": "2023-01",
    "max": "2023-06"
  },
  "plan_counts": {
    "Free": 2334,
    "Premium": 786
  }
}


In [11]:
# 📈 월별 매출 추이
monthly = df.groupby("month", as_index=False)["revenue_num"].sum().sort_values("month")
plt.figure(figsize=(7,4))
plt.plot(monthly["month"], monthly["revenue_num"], marker="o", color="#1DB954")
plt.title("Monthly Revenue (₩)")
plt.xlabel("Month"); plt.ylabel("Revenue")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(ART_FIG / "monthly_revenue.png", dpi=144)
plt.close()

# 📊 최신월 요금제별 사용자수
latest = df["month"].max()
users_mix = (
    df[df["month"]==latest]
    .groupby("subscription_plan_norm")["userid"].nunique()
    .reset_index(name="users").sort_values("users", ascending=False)
)
plt.figure(figsize=(6,4))
plt.bar(users_mix["subscription_plan_norm"], users_mix["users"], color=["#1DB954" if "rem" in p.lower() else "#BFBFBF" for p in users_mix["subscription_plan_norm"]])
plt.title(f"Active Users by Plan — {latest}")
for i, v in enumerate(users_mix["users"]):
    plt.text(i, v, f"{int(v):,}", ha="center", va="bottom")
plt.tight_layout()
plt.savefig(ART_FIG / "users_by_plan_latest.png", dpi=144)
plt.close()


In [12]:
out_path = PROC / "spotify_processed.parquet"
df.to_parquet(out_path, index=False)
print("저장 완료:", out_path)

저장 완료: /Users/yujoolee/project/StayOrSkip/data/processed/spotify_processed.parquet


In [13]:
# ---- AARRR helper: 이전/현재 플랜, 코호트, 월별 사용자 수/매출 ----
@st.cache_data(show_spinner=False)
def build_aarr_data(df: pd.DataFrame):
    d = df.copy()
    # 정렬
    d["month_key"] = d["month"].str.replace("-", "").astype(int)
    d = d.sort_values(["userid", "month_key"])

    # 이전/현재 플랜
    d["prev_plan"] = d.groupby("userid")["subscription_plan_norm"].shift(1)
    d["curr_plan"] = d["subscription_plan_norm"]

    # 전환/유지/이탈 플래그
    d["conv_fp"]   = (d["prev_plan"].str.contains("ree", case=False, na=False)) & (d["curr_plan"].str.contains("rem", case=False, na=False))
    d["ret_pp"]    = (d["prev_plan"].str.contains("rem", case=False, na=False)) & (d["curr_plan"].str.contains("rem", case=False, na=False))
    d["churn_pf"]  = (d["prev_plan"].str.contains("rem", case=False, na=False)) & (d["curr_plan"].str.contains("ree", case=False, na=False))

    # 월별 분모/분자
    bym = d.groupby("month", as_index=False).agg(
        free_prev   = ("prev_plan", lambda s: s.str.contains("ree",  case=False, na=False).sum()),
        prem_prev   = ("prev_plan", lambda s: s.str.contains("rem",  case=False, na=False).sum()),
        conv_cnt    = ("conv_fp",  "sum"),
        ret_cnt     = ("ret_pp",   "sum"),
        churn_cnt   = ("churn_pf", "sum"),
        users       = ("userid",   pd.Series.nunique),
        revenue_sum = ("revenue_num", "sum"),
    )
    # 비율
    bym["conv_rate"]  = (bym["conv_cnt"]  / bym["free_prev"]).replace([np.inf, np.nan], 0.0)
    bym["ret_rate"]   = (bym["ret_cnt"]   / bym["prem_prev"]).replace([np.inf, np.nan], 0.0)
    bym["churn_rate"] = (bym["churn_cnt"] / bym["prem_prev"]).replace([np.inf, np.nan], 0.0)
    bym["arpu"]       = bym["revenue_sum"] / bym["users"]

    # 코호트: 첫 등장 월
    first_month = d.groupby("userid", as_index=False)["month"].min().rename(columns={"month": "first_month"})
    d = d.merge(first_month, on="userid", how="left")
    d["cohort_age"] = pd.Categorical(d["month"]).codes - pd.Categorical(d["first_month"]).codes  # 0,1,2,...

    # 코호트 피벗 (유지율: 해당 코호트 첫달 사용자 대비 해당 월 활성이 있는 비율)
    base = d.groupby("first_month")["userid"].nunique().rename("cohort_size")
    active = d.groupby(["first_month","cohort_age"])["userid"].nunique().rename("active_users").reset_index()
    coh = active.merge(base, on="first_month", how="left")
    coh["retention"] = (coh["active_users"] / coh["cohort_size"]).fillna(0.0)
    coh_pivot = coh.pivot(index="first_month", columns="cohort_age", values="retention").fillna(0.0)

    return d, bym.sort_values("month"), coh_pivot

wide, monthly, cohort = build_aarr_data(df)


NameError: name 'st' is not defined